In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")  

prompt = "A image of futuristic city"


with torch.autocast("cuda"):
    image = pipe(prompt).images[0]

output_path = "generated_image.png"
image.save(output_path)

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from diffusers import StableDiffusionPipeline
import torch
import torchaudio

whisper_model_path = "S2A\Codes\whisper_finetuned.pkl"
processor = WhisperProcessor.from_pretrained(whisper_model_path)
model = WhisperForConditionalGeneration.from_pretrained(whisper_model_path)
model = model.to("cuda")

def transcribe_audio(audio_path):
    audio, rate = torchaudio.load(audio_path)
    if rate != 16000:
        audio = torchaudio.transforms.Resample(orig_freq=rate, new_freq=16000)(audio)
    input_features = processor(audio.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_features
    input_features = input_features.to("cuda")
    predicted_ids = model.generate(input_features)
    return processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

stable_diffusion_model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(stable_diffusion_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

def generate_image(prompt, output_path):
    with torch.autocast("cuda"):
        image = pipe(prompt).images[0]
    image.save(output_path)

audio_file = "samaudio.wav"
output_image_path = "outimg.png"

if __name__ == "__main__":
    transcribed_text = transcribe_audio(audio_file)
    generate_image(transcribed_text, output_image_path)


In [ ]:
import sounddevice as sd
import numpy as np
import torch
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from diffusers import StableDiffusionPipeline
import io

whisper_model_path = "S2A\Codes\whisper_finetuned.pkl"
processor = WhisperProcessor.from_pretrained(whisper_model_path)
model = WhisperForConditionalGeneration.from_pretrained(whisper_model_path)
model = model.to("cuda")

stable_diffusion_model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(stable_diffusion_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

def record_audio(duration=10, sample_rate=16000):
    print("Recording...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()
    print("Recording finished.")
    return audio.flatten()

def transcribe_audio(audio_data):
    input_features = processor(audio_data.numpy(), sampling_rate=16000, return_tensors="pt").input_features
    input_features = input_features.to("cuda")
    predicted_ids = model.generate(input_features)
    return processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

def generate_image(prompt, output_path):
    with torch.autocast("cuda"):
        image = pipe(prompt).images[0]
    image.save(output_path)

if __name__ == "__main__":
    audio_data = record_audio(duration=10)
    audio_data = torch.tensor(audio_data, dtype=torch.float32)
    transcribed_text = transcribe_audio(audio_data)
    print(f"Transcribed Text: {transcribed_text}")
    generate_image(transcribed_text, "otman_img.png")


Recording...
Recording finished.
Transcribed Text: a man in a forest
